In [2]:
#textFile = sc.textFile("file:///home/hadoop/Jiaoyan/ds504-project/data/user_log.csv")
#sample = textFile.sample(False, 0.5, 12345)
#sample.saveAsTextFile("file:///home/hadoop/Jiaoyan/ds504-project/data/sample")

In [1]:
from pyspark.sql import Row
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [8]:
filename = "file:///home/hadoop/Jiaoyan/ds504-project/data/sample/part-00000"
#textFile = sc.textFile(filename)
#header = textFile.take(1)[0]
#parsedData = textFile.filter(lambda line: line != header).map(lambda line: line.split(','))

textFile = sc.textFile(filename).map(lambda t: t.split(","))

# delete province
part = textFile.map(lambda t: (t[0],t[1],t[2],t[3],t[4],t[5],t[6],t[7],t[8],t[9]))
part.collect()

[('328862', '323294', '833', '2882', '2661', '08', '29', '0', '0', '1'),
 ('328862', '996875', '1271', '2882', '2661', '08', '29', '0', '1', '1'),
 ('328862', '575153', '1271', '2882', '2661', '08', '29', '0', '0', '0'),
 ('328862', '239288', '602', '420', '4953', '08', '01', '0', '6', '0'),
 ('328862', '950862', '962', '4605', '7622', '08', '01', '0', '1', '0'),
 ('328862', '1064265', '614', '4605', '7622', '08', '01', '0', '0', '0'),
 ('328862', '809148', '602', '420', '4953', '08', '01', '0', '5', '0'),
 ('328862', '197239', '602', '420', '4953', '08', '01', '0', '3', '0'),
 ('328862', '886633', '1505', '3828', '1446', '05', '26', '0', '0', '2'),
 ('328862', '81766', '614', '4605', '7622', '07', '09', '2', '1', '0'),
 ('328862', '81766', '614', '4605', '7622', '07', '09', '0', '5', '2'),
 ('328862', '853901', '267', '3073', '8126', '08', '05', '2', '7', '0'),
 ('328862', '963870', '267', '3073', '8126', '08', '05', '2', '7', '2'),
 ('328862', '208525', '267', '3073', '8126', '08', '

In [9]:
# convert rdd to spark dataframe
data = part.map(lambda p : Row(user_id = p[0], item_id = p[1], cat_id = p[2], merchant_id = p[3], brand_id = p[4], \
                               month = p[5], day = p[6], action = p[7], age_range = int(p[8]), gender = p[9]))

# Infer the schema, and register the DataFrame as a table.
schemaData = spark.createDataFrame(data)
schemaData.createOrReplaceTempView("data")

schemaData.show()

+------+---------+--------+------+---+------+-------+-----------+-----+-------+
|action|age_range|brand_id|cat_id|day|gender|item_id|merchant_id|month|user_id|
+------+---------+--------+------+---+------+-------+-----------+-----+-------+
|     0|        0|    2661|   833| 29|     1| 323294|       2882|   08| 328862|
|     0|        1|    2661|  1271| 29|     1| 996875|       2882|   08| 328862|
|     0|        0|    2661|  1271| 29|     0| 575153|       2882|   08| 328862|
|     0|        6|    4953|   602| 01|     0| 239288|        420|   08| 328862|
|     0|        1|    7622|   962| 01|     0| 950862|       4605|   08| 328862|
|     0|        0|    7622|   614| 01|     0|1064265|       4605|   08| 328862|
|     0|        5|    4953|   602| 01|     0| 809148|        420|   08| 328862|
|     0|        3|    4953|   602| 01|     0| 197239|        420|   08| 328862|
|     0|        0|    1446|  1505| 26|     2| 886633|       3828|   05| 328862|
|     2|        1|    7622|   614| 09|  

In [10]:
# generate new features
# 1 Monthly click/add/favorite/purchase count for certain brand/ item/ category/ merchant
import pyspark.sql.functions as func
from pyspark.sql.functions import *
for_merchant = schemaData.groupBy(schemaData['merchant_id'],schemaData['month'], schemaData['action']).\
                            agg(count('action').alias("monthly_action_count_for_merchant"))
for_brand = schemaData.groupBy(schemaData['brand_id'],schemaData['month'], schemaData['action']).\
                            agg(count('action').alias("monthly_action_count_for_brand"))
for_category = schemaData.groupBy(schemaData['cat_id'],schemaData['month'], schemaData['action']).\
                            agg(count('action').alias("monthly_action_count_for_cat"))
for_item = schemaData.groupBy(schemaData['item_id'],schemaData['month'], schemaData['action']).\
                            agg(count('action').alias("monthly_action_count_for_item"))


addFeature1 = schemaData.join(for_merchant,["merchant_id","month","action"], "left_outer")
addFeature1 = addFeature1.join(for_brand,["brand_id","month","action"], "left_outer")
addFeature1 = addFeature1.join(for_category,["cat_id","month","action"], "left_outer")
addFeature1 = addFeature1.join(for_item,["item_id","month","action"], "left_outer")
addFeature1.show()

+-------+-----+------+------+--------+-----------+---------+---+------+-------+---------------------------------+------------------------------+----------------------------+-----------------------------+
|item_id|month|action|cat_id|brand_id|merchant_id|age_range|day|gender|user_id|monthly_action_count_for_merchant|monthly_action_count_for_brand|monthly_action_count_for_cat|monthly_action_count_for_item|
+-------+-----+------+------+--------+-----------+---------+---+------+-------+---------------------------------+------------------------------+----------------------------+-----------------------------+
|1002367|   07|     0|  1553|    5376|       4173|        6| 03|     2| 108377|                              324|                           324|                          39|                            1|
|1003178|   11|     2|  1213|    2416|       2567|        1| 11|     2| 340106|                                2|                             2|                         233|           

def toCSVLine(data):
    return ','.join(str(d) for d in data)

unionHeaderRDD = sc.parallelize( [( 'ID','DT_KEY','Grade','Score','TRF_Age' )]).union( MyRDD )

unionHeaderRDD.coalesce( 1 ).map( toCSVLine ).saveAsTextFile("MyFileLocation" )

In [12]:
#addFeature1.coalesce(1).write.csv("file:///home/hadoop/Jiaoyan/ds504-project/output.csv")

In [13]:
#2. User’s monthly action count of click/add/favorite/purchase for all merchant
for_user = schemaData.groupBy(schemaData['user_id'],schemaData['month'], schemaData['action']).\
                            agg(count('merchant_id').alias('user_monthly_action'))
addFeature2 = addFeature1.join(for_user,['user_id','month','action'], 'left_outer')
addFeature2.show()

+-------+-----+------+-------+------+--------+-----------+---------+---+------+---------------------------------+------------------------------+----------------------------+-----------------------------+-------------------+
|user_id|month|action|item_id|cat_id|brand_id|merchant_id|age_range|day|gender|monthly_action_count_for_merchant|monthly_action_count_for_brand|monthly_action_count_for_cat|monthly_action_count_for_item|user_monthly_action|
+-------+-----+------+-------+------+--------+-----------+---------+---+------+---------------------------------+------------------------------+----------------------------+-----------------------------+-------------------+
| 107943|   11|     2| 869295|   106|    7319|       2318|        1| 11|     0|                               22|                            28|                          22|                            1|                  2|
| 107943|   11|     2|  20721|   352|    7565|        455|        3| 11|     2|                         

In [14]:
#3.Days of a certains user clicked/added/favorited/purchased items in each month.
action_days = schemaData.groupBy(schemaData['user_id'],schemaData['action']).\
                            agg(count('day').alias('user_action_days'))
addFeature3 = addFeature2.join(action_days,['user_id','action'], 'left_outer')
addFeature3.show()

+-------+------+-----+-------+------+--------+-----------+---------+---+------+---------------------------------+------------------------------+----------------------------+-----------------------------+-------------------+----------------+
|user_id|action|month|item_id|cat_id|brand_id|merchant_id|age_range|day|gender|monthly_action_count_for_merchant|monthly_action_count_for_brand|monthly_action_count_for_cat|monthly_action_count_for_item|user_monthly_action|user_action_days|
+-------+------+-----+-------+------+--------+-----------+---------+---+------+---------------------------------+------------------------------+----------------------------+-----------------------------+-------------------+----------------+
| 110787|     0|   06| 935098|  1213|    4928|       3551|        1| 04|     1|                               58|                            58|                         759|                            2|                 34|              76|
| 110787|     0|   06| 935098|  1213

In [17]:
#4.Total number of unique items/brand/category for a merchant
unique_items_for_merchant = schemaData.groupBy(schemaData['merchant_id']).\
                            agg(func.countDistinct('item_id').alias('unique_items_for_merchant'))
unique_cats_for_merchant = schemaData.groupBy(schemaData['merchant_id']).\
                            agg(func.countDistinct('cat_id').alias('unique_cats_for_merchant'))
unique_brands_for_merchant = schemaData.groupBy(schemaData['merchant_id']).\
                            agg(func.countDistinct('brand_id').alias('unique_brands_for_merchant'))

addFeature4 = addFeature3.join(unique_items_for_merchant,['merchant_id'], 'left_outer')
addFeature4 = addFeature4.join(unique_cats_for_merchant,['merchant_id'], 'left_outer')
addFeature4 = addFeature4.join(unique_brands_for_merchant,['merchant_id'], 'left_outer')
addFeature4.show()

+-----------+-------+------+-----+-------+------+--------+---------+---+------+---------------------------------+------------------------------+----------------------------+-----------------------------+-------------------+----------------+-------------------------+------------------------+--------------------------+
|merchant_id|user_id|action|month|item_id|cat_id|brand_id|age_range|day|gender|monthly_action_count_for_merchant|monthly_action_count_for_brand|monthly_action_count_for_cat|monthly_action_count_for_item|user_monthly_action|user_action_days|unique_items_for_merchant|unique_cats_for_merchant|unique_brands_for_merchant|
+-----------+-------+------+-----+-------+------+--------+---------+---+------+---------------------------------+------------------------------+----------------------------+-----------------------------+-------------------+----------------+-------------------------+------------------------+--------------------------+
|       1090| 414485|     0|   11| 763778| 

In [19]:
#5.The number of unique merchant that the user clicked/purchased/added/favourites in each month
unique_merchants_in_each_month = schemaData.groupBy(schemaData['user_id'],schemaData['month'],schemaData['action'])\
                            .agg(func.countDistinct('merchant_id').alias('unique_merchants_in_each_month'))

addFeature5 = addFeature4.join(unique_merchants_in_each_month,['user_id','month','action'], 'left_outer')
addFeature5.show()

+-------+-----+------+-----------+-------+------+--------+---------+---+------+---------------------------------+------------------------------+----------------------------+-----------------------------+-------------------+----------------+-------------------------+------------------------+--------------------------+------------------------------+
|user_id|month|action|merchant_id|item_id|cat_id|brand_id|age_range|day|gender|monthly_action_count_for_merchant|monthly_action_count_for_brand|monthly_action_count_for_cat|monthly_action_count_for_item|user_monthly_action|user_action_days|unique_items_for_merchant|unique_cats_for_merchant|unique_brands_for_merchant|unique_merchants_in_each_month|
+-------+-----+------+-----------+-------+------+--------+---------+---+------+---------------------------------+------------------------------+----------------------------+-----------------------------+-------------------+----------------+-------------------------+------------------------+---------

In [20]:
#6. The number of unique user who clicked/purchased/added/favourites for a given items/brands/categories/merchant
unique_users_for_item = schemaData.groupBy(schemaData['item_id'],schemaData['action']).\
                        agg(func.countDistinct('user_id').alias('unique_users_for_item'))
    
unique_users_for_cat = schemaData.groupBy(schemaData['cat_id'],schemaData['action']).\
                        agg(func.countDistinct('user_id').alias('unique_users_for_cat'))
    
unique_users_for_brand= schemaData.groupBy(schemaData['brand_id'],schemaData['action']).\
                        agg(func.countDistinct('user_id').alias('unique_users_for_brand'))
    
unique_users_for_merchant = schemaData.groupBy(schemaData['merchant_id'],schemaData['action']).\
                        agg(func.countDistinct('user_id').alias('unique_users_for_merchant'))

addFeature6 = addFeature5.join(unique_users_for_item,['item_id','action'], 'left_outer')
addFeature6 = addFeature6.join(unique_users_for_cat,['cat_id','action'], 'left_outer')
addFeature6 = addFeature6.join(unique_users_for_brand,['brand_id','action'], 'left_outer')
addFeature6 = addFeature6.join(unique_users_for_merchant,['merchant_id','action'], 'left_outer')
addFeature6.show()

+-----------+------+--------+------+-------+-------+-----+---------+---+------+---------------------------------+------------------------------+----------------------------+-----------------------------+-------------------+----------------+-------------------------+------------------------+--------------------------+------------------------------+---------------------+--------------------+----------------------+-------------------------+
|merchant_id|action|brand_id|cat_id|item_id|user_id|month|age_range|day|gender|monthly_action_count_for_merchant|monthly_action_count_for_brand|monthly_action_count_for_cat|monthly_action_count_for_item|user_monthly_action|user_action_days|unique_items_for_merchant|unique_cats_for_merchant|unique_brands_for_merchant|unique_merchants_in_each_month|unique_users_for_item|unique_users_for_cat|unique_users_for_brand|unique_users_for_merchant|
+-----------+------+--------+------+-------+-------+-----+---------+---+------+---------------------------------+---

In [21]:
#7.Counts of clicked/purchased/added/favorited of items for a certain user on the last Double 11 day
filtered = schemaData.filter((schemaData['month'] == 11) & (schemaData['day'] == 11))
action_on_double11 = filtered.groupBy(filtered['user_id'],filtered['action']).\
                        agg(func.countDistinct('action').alias('action_on_double11'))
addFeature7 = addFeature6.join(action_on_double11,['user_id','action'], 'left_outer')
addFeature7 = addFeature7.na.fill(0)
addFeature7.show()

+-------+------+-----------+--------+------+-------+-----+---------+---+------+---------------------------------+------------------------------+----------------------------+-----------------------------+-------------------+----------------+-------------------------+------------------------+--------------------------+------------------------------+---------------------+--------------------+----------------------+-------------------------+------------------+
|user_id|action|merchant_id|brand_id|cat_id|item_id|month|age_range|day|gender|monthly_action_count_for_merchant|monthly_action_count_for_brand|monthly_action_count_for_cat|monthly_action_count_for_item|user_monthly_action|user_action_days|unique_items_for_merchant|unique_cats_for_merchant|unique_brands_for_merchant|unique_merchants_in_each_month|unique_users_for_item|unique_users_for_cat|unique_users_for_brand|unique_users_for_merchant|action_on_double11|
+-------+------+-----------+--------+------+-------+-----+---------+---+------

In [23]:
#8.1 The number of users who bought on at least two different days from items
filtered_purchase =  schemaData.filter(schemaData['action'] == 2)
purchase_item = filtered_purchase.groupBy(filtered_purchase['item_id'],filtered_purchase['user_id']).\
                            agg(func.countDistinct('day').alias('purchase_days'))
purchase_item_two_times = purchase_item.filter(purchase_item['purchase_days'] > 1)
user_purchase_item_two_times = purchase_item_two_times.groupBy(purchase_item_two_times['item_id']).\
                            agg(func.countDistinct('user_id').alias('user_purchase_item_two_times'))

join_item_purchase = addFeature7.join(user_purchase_item_two_times,['item_id'], 'left_outer')
join_item_purchase = join_item_purchase.na.fill(0)

#8.2 The number of users who bought on at least two different days from brands
purchase_brand = filtered_purchase.groupBy(filtered_purchase['brand_id'],filtered_purchase['user_id']).\
                            agg(func.countDistinct('day').alias('purchase_days'))
purchase_brand_two_times = purchase_brand.filter(purchase_brand['purchase_days'] > 1)
user_purchase_brand_two_times = purchase_brand_two_times.groupBy(purchase_brand_two_times['brand_id']).\
                            agg(func.countDistinct('user_id').alias('user_purchase_brand_two_times'))

join_brand_purchase = join_item_purchase.join(user_purchase_brand_two_times,['brand_id'], 'left_outer')
join_brand_purchase = join_brand_purchase.na.fill(0)

#8.3 The number of users who bought on at least two different days from categories
purchase_cat = filtered_purchase.groupBy(filtered_purchase['cat_id'],filtered_purchase['user_id']).\
                            agg(func.countDistinct('day').alias('purchase_days'))
purchase_cat_two_times = purchase_cat.filter(purchase_cat['purchase_days'] > 1)
user_purchase_cat_two_times = purchase_cat_two_times.groupBy(purchase_cat_two_times['cat_id']).\
                            agg(func.countDistinct('user_id').alias('user_purchase_cat_two_times'))

join_cat_purchase = join_brand_purchase.join(user_purchase_cat_two_times,['cat_id'], 'left_outer')
join_cat_purchase = join_cat_purchase.na.fill(0)

#8.4 The number of users who bought on at least two different days from merchants
purchase_merchant = filtered_purchase.groupBy(filtered_purchase['merchant_id'],filtered_purchase['user_id']).\
                            agg(func.countDistinct('day').alias('purchase_days'))
purchase_merchant_two_times = purchase_merchant.filter(purchase_merchant['purchase_days'] > 1)
user_purchase_merchant_two_times = purchase_merchant_two_times.groupBy(purchase_merchant_two_times['merchant_id']).\
                            agg(func.countDistinct('user_id').alias('user_purchase_merchant_two_times'))

join_merchant_purchase = join_cat_purchase.join(user_purchase_merchant_two_times,['merchant_id'], 'left_outer')
join_merchant_purchase = join_merchant_purchase.na.fill(0)

join_merchant_purchase.show()

+-----------+------+--------+-------+-------+------+-----+---------+---+------+---------------------------------+------------------------------+----------------------------+-----------------------------+-------------------+----------------+-------------------------+------------------------+--------------------------+------------------------------+---------------------+--------------------+----------------------+-------------------------+------------------+----------------------------+-----------------------------+---------------------------+--------------------------------+
|merchant_id|cat_id|brand_id|item_id|user_id|action|month|age_range|day|gender|monthly_action_count_for_merchant|monthly_action_count_for_brand|monthly_action_count_for_cat|monthly_action_count_for_item|user_monthly_action|user_action_days|unique_items_for_merchant|unique_cats_for_merchant|unique_brands_for_merchant|unique_merchants_in_each_month|unique_users_for_item|unique_users_for_cat|unique_users_for_brand|uniq

In [24]:
join_merchant_purchase.coalesce(1).write.csv("file:///home/hadoop/Jiaoyan/ds504-project/output.csv")

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 37844)


Traceback (most recent call last):
  File "/usr/lib/python3.5/socketserver.py", line 313, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.5/socketserver.py", line 341, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.5/socketserver.py", line 354, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.5/socketserver.py", line 681, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pyspark/serializers.py", line 545, in read_int
    raise EOFError
EOFError


----------------------------------------
